Code for **super-resolution** (figures $1$ and $5$ from main paper).. Change `factor` to $8$ to reproduce images from fig. $9$ from supmat.

You can play with parameters and see how they affect the result. 

# Import libs

In [3]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import argparse
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models import *

import torch
import torch.optim

from skimage.metrics import peak_signal_noise_ratio
from models.downsampler import Downsampler

from utils.sr_utils import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

imsize = -1 
factor = 2 # 8
enforse_div32 = 'CROP' # we usually need the dimensions to be divisible by a power of two (32 in this case)
PLOT = True

# To produce images from the paper we took *_GT.png images from LapSRN viewer for corresponding factor,
# e.g. x4/zebra_GT.png for factor=4, and x8/zebra_GT.png for factor=8 
path_to_image = 'sr_data/test2_HR.bmp'
path_to_beam = 'x_8_8.npy'

# Load image and baselines

In [4]:
# # Starts here
# imgs = load_LR_HR_imgs_sr(path_to_image , imsize, factor, enforse_div32)
# # 
# # imgs['bicubic_np'], imgs['sharp_np'], imgs['nearest_np'] = get_baselines(imgs['LR_pil'], imgs['HR_pil'])

# if PLOT:
#     plot_image_grid([imgs['HR_np'], imgs['bicubic_np'], imgs['sharp_np'], imgs['nearest_np']], 4,12);
#     print ('PSNR bicubic: %.4f   PSNR nearest: %.4f' %  (
#                                         peak_signal_noise_ratio(imgs['HR_np'], imgs['bicubic_np']), 
#                                         peak_signal_noise_ratio(imgs['HR_np'], imgs['nearest_np'])))

# Set up parameters and net

In [5]:
input_depth = 4
 
INPUT =     'noise'
pad   =     'reflection'
OPT_OVER =  'net'
KERNEL_TYPE='lanczos'

LR = 0.01
tv_weight = 0.0

OPTIMIZER = 'adam'

if factor == 4: 
    num_iter = 2000
    reg_noise_std = 0.03
elif factor == 8:
    num_iter = 4000
    reg_noise_std = 0.05
elif factor == 3:
    num_iter = 1500
    reg_noise_std = 0.025
elif factor == 2:
    num_iter = 1000
    reg_noise_std = 0.02
else:
    assert False, 'We did not experiment with other factors'

# functions

In [6]:
# def calculate_SINR(H,interference,bestbeam,codebook_square,Noise_power):
#     Number_time = H.shape[1]
#     SINR = np.zeros(Number_time)
#     for index_time in range(Number_time):
#         weight = codebook_square[-1,int(bestbeam[index_time,0]),int(bestbeam[index_time,1]),:]
#         signal_power = np.abs(np.dot(weight,np.conjugate(H[:,index_time])))**2
#         interference_power = np.abs(np.dot(weight,np.conjugate(interference)))**2
#         SINR[index_time] = signal_power / (Noise_power + interference_power)
#     return (SINR)

# def calculate_SINR_first(H,interference,bestbeam,codebook_square,Noise_power):
#     Number_time = H.shape[1]
#     SINR = np.zeros(Number_time)
#     for index_time in range(Number_time):
#         weight = codebook_square[0,int(bestbeam[index_time,0]),int(bestbeam[index_time,1]),:]
#         signal_power = np.abs(np.dot(weight,np.conjugate(H[:,index_time])))**2
#         interference_power = np.abs(np.dot(weight,np.conjugate(interference)))**2
#         SINR[index_time] = signal_power / (Noise_power + interference_power)
#     return (SINR)


# def calculate_SINR_simple(H,interference,weight,Noise_power):
#     signal_power = np.abs(np.dot(weight,np.conjugate(H)))**2
#     interference_power = np.abs(np.dot(weight,np.conjugate(interference)))**2
#     SNR = signal_power / (Noise_power + interference_power)
#     return (SNR)

# def hierarchical_measurement_SINR(H, interference, N, Codebook, Number_list_beams_v, Number_list_beams_h):
#     Number_stage = Number_list_beams_h.size
#     Number_time = H.shape[1]
#     Label_square = np.zeros((Number_time))
#     Received_power = np.zeros((Number_time,Number_stage,int(Number_list_beams_v[-1]),int(Number_list_beams_h[-1])), dtype=np.float)
#     for index_time in range(Number_time):
#         for index_stage, number_beams_v,  number_beams_h in zip(range(Number_stage), Number_list_beams_v, Number_list_beams_h):  
#             for index_v in range(number_beams_v):
#                 for index_h in range(number_beams_h):
#                     y = H[:,index_time] + N
#                     weight = Codebook[index_stage, index_v,index_h,:]
#                     if index_stage == 1:
#                         y = H[:,index_time]
#                         interference_power = np.abs(np.dot(weight,np.conjugate(interference)))**2
#                         Received_power[index_time,index_stage,index_v,index_h] = np.abs(np.dot(weight,np.conjugate(y))) / interference_power
#                     else:
#                         Received_power[index_time,index_stage,index_v,index_h] = np.abs(np.dot(weight,np.conjugate(y)))
#         bestbeam_hierarchical = np.unravel_index(np.argmax(Received_power_square[index_time,-1,:,:], axis=None), Received_power_square[index_time,-1,:,:].shape)
#         Label_square = bestbeam_hierarchical[-1]*Number_list_beams_h[-1]+bestbeam_hierarchical[1]
#     return(Received_power,bestbeam_hierarchical)

# def inner_angle(A,B):
#     x = np.inner(A,B)
#     s = np.linalg.norm(A)
#     t = np.linalg.norm(B)
#     angle = np.arccos(x/(s*t))*180/pi
#     return(angle)
    
# def SINR_all(H_all, interferece_all, Noise_power, codebook, predict_classes, indices_test):
#     Number_time = H_all.shape[2]
#     SINR = np.zeros(indices_test.shape[0]*Number_time)
    
#     Number_v = codebook.shape[1]
#     Number_h = codebook.shape[2]
#     predicts_v = np.divmod(predict_classes, Number_h)[0]
#     predicts_h = np.divmod(predict_classes, Number_h)[1]
#     add = np.matlib.repmat(np.array(range(Number_time)),1,indices_test.shape[0]).T
#     indices_sim = np.kron(indices_test,np.ones(Number_time)).reshape(indices_test.shape[0]*Number_time,1)
#     indices_time = indices_sim + add
#     sinr_index = 0
#     for index_time, index_sim, predict_v, predict_h in zip(indices_time,indices_sim,predicts_v,predicts_h):
#         weight = codebook[-1, predict_v, predict_h, :]
#         index_tti = int(index_time - index_sim)
#         SINR[sinr_index] = calculate_SINR_simple(H_all[:,int(index_sim),index_tti],interference_all[:,int(index_sim)],weight,Noise_power)
#         sinr_index = sinr_index + 1
#     return(SINR)

# def SINR_all_LSTM(H_all, interferece_all, Noise_power, codebook, predict_classes, indices_test, index_LSTM):
#     Number_time = index_LSTM.shape[0]
#     SINR = np.zeros(indices_test.shape[0]*Number_time)
    
#     Number_v = codebook.shape[1]
#     Number_h = codebook.shape[2]
#     predicts_v = np.divmod(predict_classes, Number_h)[0]
#     predicts_h = np.divmod(predict_classes, Number_h)[1]
#     add = np.matlib.repmat(index_LSTM,1,indices_test.shape[0]).T
#     indices_sim = np.kron(indices_test,np.ones(Number_time)).reshape(indices_test.shape[0]*Number_time,1)
#     indices_time = indices_sim + add
#     sinr_index = 0
#     for index_time, index_sim, predict_v, predict_h in zip(indices_time,indices_sim,predicts_v,predicts_h):
#         weight = codebook[-1, int(predict_v), int(predict_h), :]
#         index_tti = int(index_time - index_sim)
#         SINR[sinr_index] = calculate_SINR_simple(H_all[:,int(index_sim),index_tti],interference_all[:,int(index_sim)],weight,Noise_power)
#         sinr_index = sinr_index + 1
#     return(SINR)

# def p_select(epsilon):
#     if epsilon > random():
#         return True
#     else:
#         return False

# def get_beam_pattern_3D_angle(M_v,M_h,weight):
#     angles_azimuth = np.arange(-90, 90, 1)
#     angles_zenith = np.arange(0, 180, 1)
#     beam_pattern = np.zeros((int(angles_zenith.size), int(angles_azimuth.size)))
#     index_v = index_h = 0
#     for angle_zenith in angles_zenith:
#         for angle_azimuth in angles_azimuth:
#             steer = (1/M_v/M_h)*np.kron(np.exp(1j*pi*np.cos(np.radians(angle_zenith))*np.arange(M_v)), np.exp(1j*pi*np.sin(np.radians(angle_zenith))*np.sin(np.radians(angle_azimuth))*np.arange(M_h)))
#             beam_pattern[index_h,index_v] = np.square(np.abs(np.dot(steer,weight)))
#             index_h = index_h + 1
#         index_v = index_v + 1
#         index_h = 0
#     return(beam_pattern, angles_zenith, angles_azimuth)

# def generate_channel(M, LOS_theta, LOS_phi):
#     Number_path =  int(LOS_theta.size)
#     H = np.zeros((M),dtype=np.complex)
#     for angle_theta, angle_phi in zip(LOS_theta, LOS_phi):
#         radian_theta = angle_theta / 180 * pi
#         radian_phi = angle_phi / 180 * pi
#         alpha = (1/np.sqrt(2))*(np.random.randn(1)+1j*np.random.randn(1))
#         steer_v = np.exp(-1j*pi*np.cos(radian_theta)*np.arange(M_v))
#         steer_h = np.exp(-1j*pi*np.sin(radian_theta)*np.sin(radian_phi)*np.arange(M_h))
#         steer_Rx = np.kron(steer_v, steer_h)
#         H = H + (alpha/np.sqrt(Number_path))*steer_Rx
#     return(H)

# def construct_codebook_packing(M, Number_list_beams, Active_M, W):
#     M_v = M_h = math.sqrt(M)
#     Number_stage = Number_list_beams.size
#     Number_beams = Number_list_beams[-1]
#     Codebook_DFT = np.zeros((int(Number_stage), int(Number_beams), int(M)),dtype=np.complex)
#     Codebook_Slepian = np.zeros((int(Number_stage), int(Number_beams), int(M)),dtype=np.complex)
#     beam2phase = np.zeros((2, int(Number_stage), int(Number_beams)))
#     index_stage = 0;
#     for number_beams, active_M , width in zip(Number_list_beams, Active_M, W):
#         w_DFT = np.ones(active_M)
#         w_DFT = (1/np.linalg.norm(w_DFT))*w_DFT
#         w_Slepian, energy = slepian(active_M, width)
        
                         
#         if number_beams == 7:
#             r = 0.333333333333*pi;
#             for index_tmp in range(6):
#                 index_beam = index_tmp + 1
#                 phase = (2*pi)/6*index_beam + pi/6
#                 phase_theta = (2*r)*np.sin(phase)
#                 phase_phi = (2*r)*np.cos(phase)
#                 shift_theta = np.exp(-1j*phase_theta*np.arange(active_M))
#                 shift_phi = np.exp(-1j*phase_phi*np.arange(active_M))
#                 beam2phase[:,index_stage,index_beam] = [phase_theta,phase_phi]
#                 #Codebook_DFT[index_stage, index_beam, :] = np.kron((np.concatenate((w_DFT*shift_theta), np.zeros(M - active_M)), axis=0),(np.concatenate((w_DFT*shift_phi), np.zeros(M - active_M)), axis=0))
#                 Codebook_DFT[index_stage, index_beam, :] = np.kron((np.concatenate((w_DFT*shift_theta, np.zeros(int(M_v - active_M))))),(np.concatenate((w_DFT*shift_phi, np.zeros(int(M_h - active_M))))))
#                 Codebook_Slepian[index_stage, index_beam, :] = np.kron((np.concatenate((w_Slepian*shift_theta, np.zeros(int(M_v - active_M))))),(np.concatenate((w_Slepian*shift_phi, np.zeros(int(M_h - active_M))))))
#         elif number_beams == 19:
#             r = 0.205604646760*pi
#             for index_tmp in range(6):
#                 index_beam = index_tmp + 1
#                 phase = (2*pi)/6*index_beam + pi/6
#                 phase_theta = (2*r)*np.sin(phase)
#                 phase_phi = (2*r)*np.cos(phase)
#                 shift_theta = np.exp(-1j*phase_theta*np.arange(active_M))
#                 shift_phi = np.exp(-1j*phase_phi*np.arange(active_M))
#                 radian_theta = np.arccos(phase_theta/pi)
#                 radian_phi = np.arcsin(phase_phi/pi/np.sin(radian_theta))
#                 beam2phase[:,index_stage,index_beam] = [np.degrees(radian_theta),np.degrees(radian_phi)]
#                 Codebook_DFT[index_stage, index_beam, :] = np.kron((np.concatenate((w_DFT*shift_theta, np.zeros(int(M_v - active_M))))),(np.concatenate((w_DFT*shift_phi, np.zeros(int(M_h - active_M))))))
#                 Codebook_Slepian[index_stage, index_beam, :] = np.kron((np.concatenate((w_Slepian*shift_theta, np.zeros(int(M_v - active_M))))),(np.concatenate((w_Slepian*shift_phi, np.zeros(int(M_h - active_M))))))
#             for index_tmp in range(12):
#                 index_beam = index_tmp + 7
#                 phase = (2*pi)/12*index_tmp + pi/12
#                 phase_theta = (2*r)*2*np.sin(phase)
#                 phase_phi = (2*r)*2*np.cos(phase)
#                 shift_theta = np.exp(-1j*phase_theta*np.arange(active_M))
#                 shift_phi = np.exp(-1j*phase_phi*np.arange(active_M))
#                 radian_theta = np.arccos(phase_theta/pi)
#                 radian_phi = np.arcsin(phase_phi/pi/np.sin(radian_theta))
#                 beam2phase[:,index_stage,index_beam] = [np.degrees(radian_theta),np.degrees(radian_phi)]
#                 Codebook_DFT[index_stage, index_beam, :] = np.kron((np.concatenate((w_DFT*shift_theta, np.zeros(int(M_v - active_M))))),(np.concatenate((w_DFT*shift_phi, np.zeros(int(M_h - active_M))))))
#                 Codebook_Slepian[index_stage, index_beam, :] = np.kron((np.concatenate((w_Slepian*shift_theta, np.zeros(int(M_v - active_M))))),(np.concatenate((w_Slepian*shift_phi, np.zeros(int(M_h - active_M))))))
                
#         shift_theta = np.exp(-1j*0*np.arange(active_M))
#         shift_phi = np.exp(-1j*0*np.arange(active_M))
#         radian_theta = np.arccos(0/pi)
#         radian_phi = np.arcsin(0/pi/np.sin(radian_theta))
#         beam2phase[:,index_stage,index_beam] = [np.degrees(radian_theta),np.degrees(radian_phi)]
#         Codebook_DFT[index_stage, 0, :] = np.kron((np.concatenate((w_DFT*shift_theta, np.zeros(int(M_v - active_M))))),(np.concatenate((w_DFT*shift_phi, np.zeros(int(M_h - active_M))))))
#         Codebook_Slepian[index_stage, 0, :] = np.kron((np.concatenate((w_Slepian*shift_theta, np.zeros(int(M_v - active_M))))),(np.concatenate((w_Slepian*shift_phi, np.zeros(int(M_h - active_M))))))
#         index_stage = index_stage + 1
    
#     return(Codebook_DFT,Codebook_Slepian,beam2phase)


# def phase2angle(phase_v, phase_h):
#     radian_theta = np.arccos(phase_v/pi)
#     radian_phi = np.arcsin(phase_h/pi/np.sin(radian_theta))
#     angle = [np.degrees(radian_theta),np.degrees(radian_phi)]
#     return(angle)


# def hierarchical_measurement(H, Noise_power, Codebook, Number_list_beams):
#     Number_stage = int(Number_list_beams.size)
#     Number_time = H.shape[1]
#     M = H.shape[0]
#     Received_power = np.zeros((Number_time,Number_stage, Number_time,int(Number_list_beams[-1])), dtype=np.float)
#     bestbeam_hierarchical = np.zeros((2,int(Number_stage)))
#     for index_time in range(Number_time):
#         for index_stage, number_beams in zip(range(Number_stage), Number_list_beams):
#             for index in range(number_beams):
#                 y = H[:,index_time] + N
#                 N = np.sqrt(Noise_power)*np.random.randn(M)
#                 Received_power[index_time,index_stage,index_time,index] = np.abs(np.dot(Codebook[index_stage, index,:],np.conjugate(y)))
#     return(Received_power)

# def exhaustive_searching_square(H, Noise_power, Codebook, Number_list_beams_v, Number_list_beams_h): 
#     Number_beams_v = Number_list_beams_v[-1]
#     Number_beams_h = Number_list_beams_h[-1]
#     Number_stage = Number_list_beams_h.size
#     Number_time = H.shape[1]
#     M = H.shape[0]
#     Received_power = np.zeros((int(Number_beams_v),int(Number_beams_h)), dtype=np.float)
#     bestbeam_exhaustive = np.zeros((Number_time,2))
#     Codebook = Codebook[-1, :, :, :]
#     for index_time in range(Number_time):
#         N = np.sqrt(Noise_power)*np.random.randn(M)
#         for index_v in range(Number_beams_v):
#             for index_h in range(Number_beams_h):
#                 y = H[:,index_time] + N
#                 Received_power[index_v,index_h] = np.abs(np.dot(Codebook[index_v,index_h,:],np.conjugate(y)))

#         bestbeam_exhaustive[index_time,:] = np.unravel_index(np.argmax(Received_power, axis=None), Received_power.shape)
#     return(bestbeam_exhaustive)


# def exhaustive_searching(H, N, Codebook, Number_list_beams): 
#     Number_beams = Number_list_beams[-1]
#     Number_stage = Number_list_beams.size
#     Received_power = np.zeros((int(Number_beams)), dtype=np.float)
#     Codebook = Codebook[-1, :, :]
    
#     for index_beam in range(Number_beams):
#         # y = H + N
#         Received_power[index_beam] = np.abs(np.dot(Codebook[index_beam,:],np.conjugate(y)))

#     bestbeam_exhaustive = np.unravel_index(np.argmax(Received_power, axis=None), Received_power.shape)
#     return(bestbeam_exhaustive)

# def construct_codebook_square(M, Number_list_beams_v, Number_list_beams_h, Active_M_v, Active_M_h, W_v, W_h, range_v, range_h):
#     M_v = M_h = math.sqrt(M)
#     Number_stage = Number_list_beams_v.size
#     Number_beams_v = Number_list_beams_v[-1]
#     Number_beams_h = Number_list_beams_h[-1]
#     Length_v = range_v[1] - range_v[0]
#     Length_h = range_h[1] - range_h[0]
#     beam2angle = np.zeros((int(Number_stage), int(Number_beams_v), int(Number_beams_h), 2))
#     Codebook_DFT = np.zeros((int(Number_stage), int(Number_beams_v), int(Number_beams_h), int(M)), dtype=np.complex)
#     Codebook_Slepian = np.zeros((int(Number_stage), int(Number_beams_v), int(Number_beams_h), int(M)), dtype=np.complex)
#     index_stage = 0;
#     for index_stage, number_beams_v, number_beams_h, active_M_v, active_M_h, width_v, width_h in zip(range(int(Number_stage)), Number_list_beams_v, Number_list_beams_h, Active_M_v, Active_M_h, W_v, W_h):
#         w_DFT_v = np.ones(active_M_v)
#         w_DFT_v = (1/np.linalg.norm(w_DFT_v))*w_DFT_v
#         w_DFT_h = np.ones(active_M_h)
#         w_DFT_h = (1/np.linalg.norm(w_DFT_h))*w_DFT_h
#         w_Slepian_v, energy = slepian(active_M_v, width_v)
#         w_Slepian_h, energy = slepian(active_M_h, width_h)
#         list_phase_v = np.linspace(range_v[0]+(Length_v/(number_beams_v*2)), range_v[1]-(Length_v/(number_beams_v*2)), num=number_beams_v)
#         list_phase_h = np.linspace(range_h[0]+(Length_h/(number_beams_h*2)), range_h[1]-(Length_h/(number_beams_h*2)), num=number_beams_h)
#         for index_v, phase_theta in zip(range(number_beams_v), list_phase_v):
#             for index_h, phase_phi in zip(range(number_beams_h), list_phase_h):
#                 shift_theta = np.exp(-1j*phase_theta*np.arange(active_M_v))
#                 shift_phi = np.exp(-1j*phase_phi*np.arange(active_M_h))
#                 Codebook_DFT[index_stage, index_v, index_h, :] = np.kron((np.concatenate((w_DFT_v*shift_theta, np.zeros(int(M_v - active_M_v))))),(np.concatenate((w_DFT_h*shift_phi, np.zeros(int(M_h - active_M_h))))))
#                 Codebook_Slepian[index_stage, index_v, index_h, :] = np.kron((np.concatenate((w_Slepian_v*shift_theta, np.zeros(int(M_v - active_M_v))))),(np.concatenate((w_Slepian_h*shift_phi, np.zeros(int(M_h - active_M_h))))))
#                 beam2angle[index_stage, index_v, index_h, 0] = np.arccos(phase_theta/pi)
#                 tmp = phase_phi/np.sin(beam2angle[index_stage, index_v, index_h, 0])/pi
#                 if tmp > 1:
#                     beam2angle[index_stage, index_v, index_h, 1] = pi / 2
#                 elif tmp < -1:
#                     beam2angle[index_stage, index_v, index_h, 1] = -pi / 2
#                 else:
#                     beam2angle[index_stage, index_v, index_h, 1] = np.arcsin(tmp)
#     print(Codebook_DFT.shape)                              
#     return(Codebook_DFT,Codebook_Slepian,beam2angle)

# def slepian(M,W):
#     A = np.zeros((M,M))
#     for j in range(M):
#         for k in range(j,M):
#             if j == k:
#                 A[j,k] = 2*W
#             else:
#                 A[j,k] = 2*np.sin(pi*W*(k-j))/(pi*(k-j))
#                 A[k,j] = 2*np.sin(pi*W*(j-k))/(pi*(j-k))

#     U,S,V = np.linalg.svd(A, full_matrices=True)
#     v = V[0,:]
#     energy = S[0]
#     return(v,energy)

# def normalize(v, axis=-1, order=2):
#     l2 = np.linalg.norm(v, ord = order, axis=axis, keepdims=True)
#     l2[l2==0] = 1
#     return v/l2

# def calculate_AOA(UE_pos,BS_pos):
#     UE_x,UE_y,UE_z = UE_pos[0],UE_pos[1],UE_pos[2]
#     phi = inner_angle([0,100],[UE_x,UE_y])
#     height = UE_z - BS_pos[2]
#     dis = np.linalg.norm((UE_pos - BS_pos))
#     theta = np.arccos(height/dis)*180/pi
#     return(theta,phi)
    
# def ray_tracing_channel(M,UE_pos,BS_pos,wavelength):
#     M_h = M_v = np.sqrt(M)
#     Number_time = UE_pos.shape[1]
#     H = np.zeros((M,Number_time),dtype=np.complex)
#     north_wall = 30
#     south_wall = -40
#     Number_reflection = 1
#     UE_y = UE_pos[1,0]
#     UE_z = UE_pos[2,0]
#     BS_pos = np.array(BS_pos)
#     reflect_gain = (10**(-6/10))
#     #reflect_gain = 1
#     for UE_x,index_time in zip(UE_pos[0,:], range(Number_time)):
#         UE_pos = [UE_x,UE_y,UE_z]
#         UE_pos = np.array(UE_pos)
#         theta, phi = calculate_AOA(UE_pos,BS_pos)
#         dis = np.linalg.norm((UE_pos - BS_pos))
#         radian_theta = theta / 180 * pi
#         radian_phi = phi / 180 * pi
#         channel_gain = (wavelength/4/pi/dis)*np.sqrt(M)*np.exp(-1j*2*pi*dis/wavelength)
#         steer_v = (1/np.sqrt(M_v))*np.exp(-1j*pi*np.cos(radian_theta)*np.arange(M_v))
#         steer_h = (1/np.sqrt(M_h))*np.exp(-1j*pi*np.sin(radian_theta)*np.sin(radian_phi)*np.arange(M_h))
#         steer_Rx = np.kron(steer_v, steer_h)
#         H[:,index_time] = H[:,index_time] + channel_gain * steer_Rx
        
#         UE_pos = [UE_x,UE_y,-1*UE_z]
#         UE_pos = np.array(UE_pos)
#         theta, phi = calculate_AOA(UE_pos,BS_pos)
#         dis = np.linalg.norm((UE_pos - BS_pos))
#         radian_theta = theta / 180 * pi
#         radian_phi = phi / 180 * pi
#         channel_gain = (wavelength/4/pi/dis)*np.sqrt(M)*np.exp(-1j*2*pi*dis/wavelength)*reflect_gain
#         steer_v = (1/np.sqrt(M_v))*np.exp(-1j*pi*np.cos(radian_theta)*np.arange(M_v))
#         steer_h = (1/np.sqrt(M_h))*np.exp(-1j*pi*np.sin(radian_theta)*np.sin(radian_phi)*np.arange(M_h))
#         steer_Rx = np.kron(steer_v, steer_h)
#         H[:,index_time] = H[:,index_time] + channel_gain * steer_Rx
        
#         UE_pos = [UE_x,(south_wall-UE_y)+south_wall,UE_z]
#         UE_pos = np.array(UE_pos)
#         theta, phi = calculate_AOA(UE_pos,BS_pos)
#         dis = np.linalg.norm((UE_pos - BS_pos))
#         radian_theta = theta / 180 * pi
#         radian_phi = phi / 180 * pi
#         channel_gain = (wavelength/4/pi/dis)*np.sqrt(M)*np.exp(-1j*2*pi*dis/wavelength)*reflect_gain
#         steer_v = (1/np.sqrt(M_v))*np.exp(-1j*pi*np.cos(radian_theta)*np.arange(M_v))
#         steer_h = (1/np.sqrt(M_h))*np.exp(-1j*pi*np.sin(radian_theta)*np.sin(radian_phi)*np.arange(M_h))
#         steer_Rx = np.kron(steer_v, steer_h)
#         #H[:,index_time] = H[:,index_time] + channel_gain * steer_Rx
        
#         UE_pos = [UE_x,(north_wall-UE_y)+north_wall,UE_z]
#         UE_pos = np.array(UE_pos)
#         theta, phi = calculate_AOA(UE_pos,BS_pos)
#         dis = np.linalg.norm((UE_pos - BS_pos))
#         radian_theta = theta / 180 * pi
#         radian_phi = phi / 180 * pi
#         channel_gain = (wavelength/4/pi/dis)*np.sqrt(M)*np.exp(-1j*2*pi*dis/wavelength)*reflect_gain
#         steer_v = (1/np.sqrt(M_v))*np.exp(-1j*pi*np.cos(radian_theta)*np.arange(M_v))
#         steer_h = (1/np.sqrt(M_h))*np.exp(-1j*pi*np.sin(radian_theta)*np.sin(radian_phi)*np.arange(M_h))
#         steer_Rx = np.kron(steer_v, steer_h)
#         #H[:,index_time] = H[:,index_time] + channel_gain * steer_Rx
        
#     return(H)

# def construct_clustering_channel(M, Number_cluster, Number_ray, angular_spread, is_LOS):
#     M_v = M_h = math.sqrt(M)
#     Number_clusters = np.random.poisson(Number_cluster-1, 1)
#     Number_cluster = int(Number_clusters[0])+1
#     H = np.zeros((M),dtype=np.complex)
#     theta_list_cluster_rx = 180*np.random.rand(Number_cluster)
#     phi_list_cluster_rx = 90*(2*np.random.rand(Number_cluster)-1)
#     gamma = math.sqrt(M/Number_cluster/Number_ray)

#     for theta_cluster_rx, phi_cluster_rx in zip(theta_list_cluster_rx, phi_list_cluster_rx):
#         theta_list_ray_rx = numpy.random.laplace(loc=0, scale=1, size=Number_ray) + theta_cluster_rx
#         phi_list_ray_rx = numpy.random.laplace(loc=0, scale=1, size=Number_ray) + phi_cluster_rx
#         for theta_ray_rx, phi_ray_rx in zip(theta_list_ray_rx, phi_list_ray_rx):
#             alpha = (1/np.sqrt(2))*(np.random.randn(1)+1j*np.random.randn(1))
#             radian_theta = theta_ray_rx / 180 * pi
#             radian_phi = phi_ray_rx / 180 * pi
#             steer_v = (1/np.sqrt(M_v))*np.exp(-1j*pi*np.cos(radian_theta)*np.arange(M_v))
#             steer_h = (1/np.sqrt(M_h))*np.exp(-1j*pi*np.sin(radian_theta)*np.sin(radian_phi)*np.arange(M_h))
#             steer_Rx = np.kron(steer_v, steer_h)
#             H = H + (alpha*gamma)*steer_Rx
    
#     if is_LOS:
#         alpha = (1/np.sqrt(2))*(np.random.randn(1)+1j*np.random.randn(1))
#         theta_ray_rx = 120*np.random.rand(1)
#         phi_ray_rx = 90*(2*np.random.rand(1)-1)
#         radian_theta = theta_ray_rx / 180 * pi
#         radian_phi = phi_ray_rx / 180 * pi
#         steer_v = (1/np.sqrt(M_v))*np.exp(-1j*pi*np.cos(radian_theta)*np.arange(M_v))
#         steer_h = (1/np.sqrt(M_h))*np.exp(-1j*pi*np.sin(radian_theta)*np.sin(radian_phi)*np.arange(M_h))
#         steer_Rx = np.kron(steer_v, steer_h)
#         LOS_H = (alpha)*steer_Rx*np.sqrt(M)
#         K_factor_dB = np.random.normal(9, 3.5, 1)
#         #K_factor_dB = 5
#         K = 10**(K_factor_dB/10)
#         K = 1000**1000
#         LOS_H = np.sqrt(K/(K+1))*LOS_H + np.sqrt(1/(K+1))*H 
#         NLOS_H = H

#     return(LOS_H, NLOS_H)

# def hierarchical_searching_square(H: np.ndarray, Noise_power: int, Codebook: np.ndarray, Number_list_beams_v: np.ndarray, Number_list_beams_h: np.ndarray):
#     Number_stage = Number_list_beams_v.size
#     Number_time = H.shape[1]
#     M = H.shape[0]
#     Received_power = np.zeros((int(Number_list_beams_v[-1]),int(Number_list_beams_h[-1])), dtype=np.float)
#     bestbeam_hierarchical = np.zeros((Number_time,2,int(Number_stage)))
#     for index_time in range(Number_time):
#         for index_stage, number_beams_v,  number_beams_h in zip(range(Number_stage), Number_list_beams_v, Number_list_beams_h):  
#             Received_power = np.zeros((number_beams_v,number_beams_h))
#             N = np.sqrt(Noise_power)*np.random.randn(M)
#             if index_stage == 0:
#                 for index_v in range(number_beams_v):
#                     for index_h in range(number_beams_h):
#                         y = H[:,index_time] + N
#                         Received_power[index_v,index_h] = np.abs(np.dot(Codebook[index_stage,index_v,index_h,:],np.conjugate(y)))
#             else:
#                 Number_measurement_v = Number_list_beams_v[index_stage] / Number_list_beams_v[index_stage-1]
#                 Number_measurement_h = Number_list_beams_h[index_stage] / Number_list_beams_h[index_stage-1]
#                 for index_measurement_v in range(int(Number_measurement_v)):
#                     for index_measurement_h in range(int(Number_measurement_h)):
#                         y = H[:,index_time] + N
#                         index_v = index_measurement_v + Number_measurement_v * bestbeam_hierarchical[index_time,0,index_stage-1]
#                         index_h = index_measurement_h + Number_measurement_h * bestbeam_hierarchical[index_time,1,index_stage-1]
#                         Received_power[int(index_v),int(index_h)] = np.abs(np.dot(Codebook[index_stage,int(index_v),int(index_h),:],np.conjugate(y)))
#             bestbeam_hierarchical[index_time,:,index_stage] = np.unravel_index(np.argmax(Received_power, axis=None), Received_power.shape)
#     return(bestbeam_hierarchical)

# def hierarchical_measurement_square(H: np.ndarray, Noise_power: int, Codebook: np.ndarray, Number_list_beams_v: np.ndarray, Number_list_beams_h: np.ndarray):
#     Number_stage = Number_list_beams_h.size
#     print("Number_stage: " + str(Number_stage))
#     Number_time = H.shape[1]
#     M = H.shape[0]
#     Label_square = np.zeros((Number_time))
#     Received_power = np.zeros((Number_time,Number_stage,int(Number_list_beams_v[-1]),int(Number_list_beams_h[-1])), dtype=np.float)
#     for index_time in range(Number_time):
#         N = np.sqrt(Noise_power)*np.random.randn(M)
#         for index_stage, number_beams_v,  number_beams_h in zip(range(Number_stage), Number_list_beams_v, Number_list_beams_h):  
#             for index_v in range(number_beams_v):
#                 for index_h in range(number_beams_h):
#                     print(N.shape)
#                     print(H.shape)
#                     y = H[:,index_time] + N
#                     # ここが肝
#                     Received_power[index_time,index_stage,index_v,index_h] = np.abs(np.dot(Codebook[index_stage, index_v,index_h,:],np.conjugate(y)))
        
#         tmp = Received_power[index_time,-1,:,:].reshape(int(Number_list_beams_v[-1]),int(Number_list_beams_h[-1]))
#         bestbeam_hierarchical = np.unravel_index(np.argmax(tmp, axis=None), tmp.shape)
#         Label_square[index_time] = int(bestbeam_hierarchical[0]*Number_list_beams_v[-1]+bestbeam_hierarchical[1])
#     return(Received_power,Label_square)


# transform beam into phase domain (LR)

In [7]:
# import math
# import numpy as np
# import numpy.matlib
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from random import random


# Number_interference = 10
# M = 64
# M_v = M_h = 8
# pi = math.pi
# k=1.38*10**(-23)
# T = 300
# Bandwidth = 120*10**(3)
# F = 10**(3/10)
# Noise_power = k*T*Bandwidth*F

# beam = np.load(path_to_beam)
# print(beam)
# #Noise_power = 0
# temp_H_all = beam*(10)
# H_all = np.zeros((temp_H_all.shape[2] ** 2, temp_H_all.shape[0], 1), dtype=np.complex128)
# for i in range(temp_H_all.shape[0]):
#   a_temp_H_all = np.ndarray.flatten(temp_H_all[i][0])
#   for j in range(a_temp_H_all.shape[0]):
#     H_all[j][i][0] = a_temp_H_all[j]


# Number_simulation = H_all.shape[1]
# Number_movement = H_all.shape[2]
# Power = np.mean((np.sum(np.abs(beam) ** 2,0)))

# Number_list_beams_v = np.array([4,8])
# Number_list_beams_h = np.array([4,8])
# Number_beams_square = Number_list_beams_v[-1] * Number_list_beams_h[-1]
# Active_M_v = np.array([3,8])
# Active_M_h = np.array([3,8])
# W_v = W_h = 1 / Active_M_v
# #W_v = W_h = np.zeros(2) + 0.0000000001
# Range_v = np.array([-1*pi, pi])
# Range_h = np.array([-1*pi, pi])
# DFT_square,Slepian_square,beam2phase = construct_codebook_square(M, Number_list_beams_v, Number_list_beams_h, Active_M_v, Active_M_h, W_v, W_h, Range_v, Range_h)
# Received_power_square = np.zeros((Number_simulation, Number_movement, 2, int(Number_list_beams_v[-1]), int(Number_list_beams_h[-1])), dtype=np.complex)
# Received_power_square2 = np.zeros((Number_simulation, Number_movement, 2, int(Number_list_beams_v[-1]), int(Number_list_beams_h[-1])), dtype=np.complex)
# Label_square = np.zeros((Number_simulation, Number_movement), dtype=np.int64)

# Capacity_DFT_exhaustive_square = np.zeros((int(Number_simulation),int(Number_movement)))
# Capacity_Slepian_exhaustive_square = np.zeros((int(Number_simulation),int(Number_movement)))
# Capacity_DFT_iterative_square = np.zeros((int(Number_simulation),int(Number_movement)))
# Capacity_Slepian_iterative_square = np.zeros((int(Number_simulation),int(Number_movement)))

# SINR_DFT_exhaustive_square = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_Slepian_exhaustive_square = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_DFT_iterative_square = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_Slepian_iterative_square = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_DFT_first_stage = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_Slepian_first_stage = np.zeros((int(Number_simulation),int(Number_movement)))
# beam_index = np.zeros((int(Number_simulation),int(Number_movement)))

# interference_all = np.zeros((M, int(Number_simulation)), dtype=np.complex)

# index_simulation = 0
# while index_simulation < Number_simulation:
#     H = H_all[:,index_simulation,:]
#     #interference
#     interference_theta = 180*np.random.rand(Number_interference)
#     interference_phi = 90*(2*np.random.rand(Number_interference)-1)
#     interference = generate_channel(M, interference_theta, interference_phi)*np.sqrt(Power)
#     interference_all[:,index_simulation] = interference
#     #Noise
#     N = np.sqrt(Noise_power)*np.random.randn(M)
    
#     #Received_power_square[index_simulation,:,:,:,:], Label_square = hierarchical_measurement_SINR(H, interference, N, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     Received_power_square[index_simulation,:,:,:,:], Label_square[index_simulation,:] = hierarchical_measurement_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     Received_power_square2[index_simulation,:,:,:,:], Label_square[index_simulation,:] = hierarchical_measurement_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     bestbeam_exhaustive_square = exhaustive_searching_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     SINR_tmp = calculate_SINR(H,interference,bestbeam_exhaustive_square, DFT_square, Noise_power)
#     SINR_Slepian_exhaustive_square[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     bestbeam_exhaustive_square = exhaustive_searching_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     SINR_tmp = calculate_SINR(H,interference,bestbeam_exhaustive_square, DFT_square, Noise_power)
#     SINR_DFT_exhaustive_square[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     bestbeam_iterative = hierarchical_searching_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     SINR_tmp = calculate_SINR(H,interference,bestbeam_iterative[:,:,-1], DFT_square, Noise_power)
#     SINR_DFT_iterative_square[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     SINR_tmp = calculate_SINR_first(H,interference,bestbeam_iterative[:,:,0], DFT_square, Noise_power)
#     SINR_DFT_first_stage[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     bestbeam_iterative = hierarchical_searching_square(H, Noise_power, Slepian_square, Number_list_beams_v, Number_list_beams_h)
#     SINR_tmp = calculate_SINR(H,interference,bestbeam_iterative[:,:,-1], Slepian_square, Noise_power)
#     SINR_Slepian_iterative_square[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     SINR_tmp = calculate_SINR_first(H,interference,bestbeam_iterative[:,:,0], Slepian_square, Noise_power)
#     SINR_Slepian_first_stage[index_simulation,:] = 10 * np.log10(SINR_tmp) 

#     index_simulation = index_simulation + 1
        
# X = Received_power_square[:,:,0,0:Number_list_beams_v[0],0:Number_list_beams_h[0]]
# Power_sharp = Received_power_square[:,:,-1,:,:]
# Power_sharp2 = Received_power_square2[:,:,-1,:,:]
# indices = np.array(range(X.shape[0]))
# #X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, Power_sharp, indices, train_size=0.8, random_state=1)


## generate HR beam image

In [8]:
# beam = np.load('y_16_16.npy')
# print(beam.shape)
# Number_interference = 10
# M = 256
# M_v = M_h = 16
# pi = math.pi
# k=1.38*10**(-23)
# T = 300
# Bandwidth = 120*10**(3)
# F = 10**(3/10)
# Noise_power = k*T*Bandwidth*F

# #Noise_power = 0
# temp_H_all = beam*(10)
# H_all = np.zeros((temp_H_all.shape[2] ** 2, temp_H_all.shape[0], 1), dtype=np.complex128)
# for i in range(temp_H_all.shape[0]):
#   a_temp_H_all = np.ndarray.flatten(temp_H_all[i][0])
#   for j in range(a_temp_H_all.shape[0]):
#     H_all[j][i][0] = a_temp_H_all[j]


# print(H_all.shape)
# Number_simulation = H_all.shape[1]
# Number_movement = H_all.shape[2]
# print(Number_movement)
# Power = np.mean((np.sum(np.abs(beam) ** 2,0)))

# Number_list_beams_v = np.array([8,16])
# Number_list_beams_h = np.array([8,16])
# Number_beams_square = Number_list_beams_v[-1] * Number_list_beams_h[-1]
# Active_M_v = np.array([7,16])
# Active_M_h = np.array([7,16])
# W_v = W_h = 1 / Active_M_v
# #W_v = W_h = np.zeros(2) + 0.0000000001
# Range_v = np.array([-1*pi, pi])
# Range_h = np.array([-1*pi, pi])
# DFT_square,Slepian_square,beam2phase = construct_codebook_square(M, Number_list_beams_v, Number_list_beams_h, Active_M_v, Active_M_h, W_v, W_h, Range_v, Range_h)
# Received_power_square = np.zeros((Number_simulation, Number_movement, 2, int(Number_list_beams_v[-1]), int(Number_list_beams_h[-1])), dtype=np.complex)
# Received_power_square2 = np.zeros((Number_simulation, Number_movement, 2, int(Number_list_beams_v[-1]), int(Number_list_beams_h[-1])), dtype=np.complex)
# Label_square = np.zeros((Number_simulation, Number_movement), dtype=np.int64)

# Capacity_DFT_exhaustive_square = np.zeros((int(Number_simulation),int(Number_movement)))
# Capacity_Slepian_exhaustive_square = np.zeros((int(Number_simulation),int(Number_movement)))
# Capacity_DFT_iterative_square = np.zeros((int(Number_simulation),int(Number_movement)))
# Capacity_Slepian_iterative_square = np.zeros((int(Number_simulation),int(Number_movement)))

# SINR_DFT_exhaustive_square = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_Slepian_exhaustive_square = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_DFT_iterative_square = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_Slepian_iterative_square = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_DFT_first_stage = np.zeros((int(Number_simulation),int(Number_movement)))
# SINR_Slepian_first_stage = np.zeros((int(Number_simulation),int(Number_movement)))
# beam_index = np.zeros((int(Number_simulation),int(Number_movement)))

# interference_all = np.zeros((M, int(Number_simulation)), dtype=np.complex)

# DFT_square = Slepian_square
# index_simulation = 0
# while index_simulation < Number_simulation:
#     H = H_all[:,index_simulation,:]
#     #interference
#     interference_theta = 180*np.random.rand(Number_interference)
#     interference_phi = 90*(2*np.random.rand(Number_interference)-1)
#     interference = generate_channel(M, interference_theta, interference_phi)*np.sqrt(Power)
#     interference_all[:,index_simulation] = interference
#     #Noise
#     N = np.sqrt(Noise_power)*np.random.randn(M)
    
#     #Received_power_square[index_simulation,:,:,:,:], Label_square = hierarchical_measurement_SINR(H, interference, N, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     Received_power_square[index_simulation,:,:,:,:], Label_square[index_simulation,:] = hierarchical_measurement_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     Received_power_square2[index_simulation,:,:,:,:], Label_square[index_simulation,:] = hierarchical_measurement_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     bestbeam_exhaustive_square = exhaustive_searching_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     SINR_tmp = calculate_SINR(H,interference,bestbeam_exhaustive_square, DFT_square, Noise_power)
#     SINR_Slepian_exhaustive_square[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     bestbeam_exhaustive_square = exhaustive_searching_square(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     SINR_tmp = calculate_SINR(H,interference,bestbeam_exhaustive_square, DFT_square, Noise_power)
#     SINR_DFT_exhaustive_square[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     bestbeam_iterative = hierarchical_searching_s畳み込みquare(H, Noise_power, DFT_square, Number_list_beams_v, Number_list_beams_h)
#     SINR_tmp = calculate_SINR(H,interference,bestbeam_iterative[:,:,-1], DFT_square, Noise_power)
#     SINR_DFT_iterative_square[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     SINR_tmp = calculate_SINR_first(H,interference,bestbeam_iterative[:,:,0], DFT_square, Noise_power)
#     SINR_DFT_first_stage[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     bestbeam_iterative = hierarchical_searching_square(H, Noise_power, Slepian_square, Number_list_beams_v, Number_list_beams_h)
#     SINR_tmp = calculate_SINR(H,interference,bestbeam_iterative[:,:,-1], Slepian_square, Noise_power)
#     SINR_Slepian_iterative_square[index_simulation,:] = 10 * np.log10(SINR_tmp)
    
#     SINR_tmp = calculate_SINR_first(H,interference,bestbeam_iterative[:,:,0], Slepian_square, Noise_power)
#     SINR_Slepian_first_stage[index_simulation,:] = 10 * np.log10(SINR_tmp) 

#     index_simulation = index_simulation + 1
        
# X = Received_power_square[:,:,0,0:Number_list_beams_v[0],0:Number_list_beams_h[0]]
# HR_Power_sharp = Received_power_square[:,:,-1,:,:]
# HR_Power_sharp2 = Received_power_square2[:,:,-1,:,:]
# indices = np.array(range(X.shape[0]))
# #X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, Power_sharp, indices, train_size=0.8, random_state=1)


In [9]:
net_input = get_noise(input_depth, INPUT, (16, 16)).type(dtype).detach()

NET_TYPE = 'skip' # UNet, ResNet

#     index_simulation = index_simulation + 1
        
# X = Received_power_square[:,:,0,0:Number_list_beams_v[0],0:Number_list_beams_h[0]]
net = get_net(input_depth, 'skip', pad,
              skip_n33d=128, 
              skip_n33u=128, 
              skip_n11=4,
              n_channels=1,
              num_scales=3,
              upsample_mode='bilinear').type(dtype)

# Losses
mse = torch.nn.MSELoss().type(dtype)
beam = np.load('x_8_8.npy')
beam = np.abs(beam)
beam = beam.astype('float32')
# img_LR_var = np_to_torch(imgs['LR_np']).type(dtype)
# print(img_LR_var.size())
# print(Power_sharp.shape)
# img_LR_var = np_to_torch(np.array([Power_sharp[0][0]])).type(dtype)
# img_HR = np_to_torch(np.array([HR_Power_sharp[0][0]])).type(dtype)
LR_beam = np.load('x_8_8.npy')
HR_beam = np.load('y_16_16.npy')
print(LR_beam.shape)
LR_max = np.max(np.array([np.abs(LR_beam[0][0])]))
HR_max = np.max(np.array([np.abs(HR_beam[0][0])]))
img_LR = np_to_torch(np.array([np.abs(LR_beam[0][0])]) / LR_max * 10e-1).type(dtype)
max_num = np.max(torch_to_np(img_LR))
img_LR_var = np_to_torch(np.array([np.abs(LR_beam[0][0])]) / LR_max * 10e-1).type(dtype)
img_HR = np_to_torch(np.array([np.abs(HR_beam[0][0])]) / HR_max * 10e-1).type(dtype)
print(img_HR)
print(img_LR)
downsampler = Downsampler(n_planes=1, factor=factor, kernel_width=4, support=1, kernel_type=KERNEL_TYPE, phase=0.5, preserve_size=True).type(dtype)
#downsampler = Downsampler(n_planes=1, factor=factor, kernel_type=KERNEL_TYPE, phase=0.5, preserve_size=True).type(dtype)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# Define closure and optimize

In [10]:
def closure():
    global i, net_input
    
    if reg_noise_std > 0:
        temp = noise.normal_() * reg_noise_std
        min_temp = torch.min(temp)
        # if (min_temp < 0):
        #   temp += min_temp
        #   print("Oh my god")
        net_input = net_input_saved + (temp)
    out_HR = net(net_input)
    out_LR = downsampler(out_HR)
    total_loss = mse(out_LR, img_LR_var)
    
    if tv_weight > 0:
        total_loss += tv_weight * tv_loss(out_HR)
        
    total_loss.backward()

    # Log
    psnr_LR = peak_signal_noise_ratio(np.array([LR_beam[0][0]]).astype(np.float32), torch_to_np(out_LR))
    # psnr_HR = peak_signal_noise_ratio(Power_sharp[0][0], torch_to_np(out_HR))
    print ('Iteration %05d    PSNR_LR %.3f  ' % (i, psnr_LR), '\r', end='')
    # History
    def resize_beam_image(out, size_of_resizing=1):
      out_np = torch_to_np(out)
      resized_out_np = np.zeros((1, out_np.shape[1] * size_of_resizing, out_np.shape[1] * size_of_resizing))
      for tempi in range(out_np.shape[1]):
        for tmpm in range(size_of_resizing):
          for tempj in range(out_np.shape[2]):
            for tempk in range(size_of_resizing):
              resized_out_np[0][tempi * size_of_resizing + tmpm][tempj * size_of_resizing + tempk] = out_np[0][tempi][tempj]
      return resized_out_np

    psnr_history.append([psnr_LR])
    if PLOT and i % 100 == 0:
        resize_beam_image(out_HR)
        out_HR_np = torch_to_np(out_HR)
        out_LR_np = torch_to_np(out_LR)
        print(i // 100, mse(img_HR, out_HR))
        print("raw LR beam image")
        plot_image_grid([resize_beam_image(img_LR, 6)], factor=4, nrow=4)
        print("temp LR beam image")
        plot_image_grid([resize_beam_image(out_LR, 6)], factor=4, nrow=1)
        print("temp HR beam image")
        plot_image_grid([resize_beam_image(out_HR, 6)], factor=4, nrow=1)
        plot_image_grid([resize_beam_image(img_HR, 6)], factor=4, nrow=1)
    i += 1
    
    return total_loss

In [11]:
psnr_history = [] 
net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()

i = 0
p = get_params(OPT_OVER, net, net_input)
optimize(OPTIMIZER, p, closure, LR, num_iter)

NameError: name 'net_input' is not defined

In [11]:
out_HR_np = np.clip(torch_to_np(net(net_input)), 0, 1)
result_deep_prior = put_in_center(out_HR_np, imgs['orig_np'].shape[1:])

# For the paper we acually took `_bicubic.png` files from LapSRN viewer and used `result_deep_prior` as our result
plot_image_grid([out_HR_np], factor=4, nrow=1);

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


NameError: ignored